In [1]:
import time
import copy
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import video
from torch.utils.data import DataLoader
import sys
sys.path.append('/home/christoph/Dokumente/christoph-MA/MA-Repo')
sys.path.append('/home/christoph/Dokumente/christoph-MA/MedicalNet/models')
sys.path.append('/home/christoph/Dokumente/christoph-MA/research-contributions/SwinUNETR')
import resnet

In [ ]:
SwinTrans = 

In [48]:

model = resnet.resnet18(sample_input_D=32,sample_input_H=128,sample_input_W=128,num_seg_classes=3) 


/home/christoph/Dokumente/christoph-MA/MedicalNet/models/resnet.py:173: FutureWarning: `nn.init.kaiming_normal` is now deprecated in favor of `nn.init.kaiming_normal_`.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')


In [ ]:
from torch.nn import DataParallel, Sequential, AdaptiveAvgPool3d, Linear

model = resnet.resnet18(sample_input_D=32, sample_input_H=128, sample_input_W=128, num_seg_classes=3) 

model = DataParallel(model) 

model = model.to('cuda')

model.module.conv_seg = Sequential(
    AdaptiveAvgPool3d(output_size=(1, 1, 1)),
    Linear(in_features=512, out_features=3, bias=True)
)

/home/christoph/Dokumente/christoph-MA/MedicalNet/models/resnet.py:173: FutureWarning: `nn.init.kaiming_normal` is now deprecated in favor of `nn.init.kaiming_normal_`.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')


In [50]:
print(model)

DataParallel(
  (module): ResNet(
    (conv1): Conv3d(1, 64, kernel_size=(7, 7, 7), stride=(2, 2, 2), padding=(3, 3, 3), bias=False)
    (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool3d(kernel_size=(3, 3, 3), stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn1): BatchNorm3d(64,

In [37]:
model_path = "/home/christoph/Dokumente/christoph-MA/MedicalNet/pretrain/resnet_18.pth"

data_path = "/storage/Datensätze"

# Load the state dictionary
state_dict = torch.load(model_path)

# Extract the nested state dictionary if it exists
if 'state_dict' in state_dict:
    state_dict = state_dict['state_dict']
print(state_dict.keys())

odict_keys(['module.conv1.weight', 'module.bn1.weight', 'module.bn1.bias', 'module.bn1.running_mean', 'module.bn1.running_var', 'module.bn1.num_batches_tracked', 'module.layer1.0.conv1.weight', 'module.layer1.0.bn1.weight', 'module.layer1.0.bn1.bias', 'module.layer1.0.bn1.running_mean', 'module.layer1.0.bn1.running_var', 'module.layer1.0.bn1.num_batches_tracked', 'module.layer1.0.conv2.weight', 'module.layer1.0.bn2.weight', 'module.layer1.0.bn2.bias', 'module.layer1.0.bn2.running_mean', 'module.layer1.0.bn2.running_var', 'module.layer1.0.bn2.num_batches_tracked', 'module.layer1.1.conv1.weight', 'module.layer1.1.bn1.weight', 'module.layer1.1.bn1.bias', 'module.layer1.1.bn1.running_mean', 'module.layer1.1.bn1.running_var', 'module.layer1.1.bn1.num_batches_tracked', 'module.layer1.1.conv2.weight', 'module.layer1.1.bn2.weight', 'module.layer1.1.bn2.bias', 'module.layer1.1.bn2.running_mean', 'module.layer1.1.bn2.running_var', 'module.layer1.1.bn2.num_batches_tracked', 'module.layer2.0.conv1

/tmp/ipykernel_4828/2631500724.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path)


In [ ]:
import torch

# Define a dictionary to store outputs
outputs = {}

# Hook function to store layer outputs
def hook_fn(module, input, output):
    outputs["layer4"] = output

# Register hook on layer4
model.module.layer4.register_forward_hook(hook_fn)

# Create a dummy input tensor matching the model's expected input shape
dummy_input = torch.randn(1, 1, 32, 128, 128).to("cuda")  # (batch, channels, depth, height, width)

# Forward pass
model(dummy_input)

# Print the captured features from layer4
print(outputs["layer4"].shape)  #
print()

torch.Size([1, 512, 4, 16, 16])


In [ ]:
#!/usr/bin/env python3
zip_file = '/home/christoph/Dokumente/christoph-MA/MedicalNet/MedicalNet_pytorch_files2.zip'
import zipfile

with zipfile.ZipFile(zip_file, "r") as zip_ref:
    zip_ref.extractall()
    